In [11]:
#imports
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from pydataset import data

import env
from sqlalchemy import text, create_engine

In [21]:
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

# Your code here

# Reset warning filter
warnings.resetwarnings()

In [4]:
# Load the 'mpg' dataset from pydataset
df = data('mpg')
df.head()

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact


In [7]:
#change trans column to auto and manual
df.trans = np.where(df.trans.str.startswith('auto'), 'auto', 'manual')

In [8]:
#split into X and y
X = df[['displ', 'year', 'cyl', 'cty', 'hwy']]
X.head()

,displ,year,cyl,cty,hwy
1,1.8,1999,4,18,29
2,1.8,1999,4,21,29
3,2.0,2008,4,20,31
4,2.0,2008,4,21,30
5,2.8,1999,6,16,26


In [9]:
#show 'y'
y = df.trans
y.head()

1      auto
2    manual
3    manual
4      auto
5      auto
Name: trans, dtype: object

## cross_val_score

In [12]:
#split into trains and tests
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state=123)

In [28]:
#get the validates
X_train, X_validate, y_train, y_validate = train_test_split(X_train,
                            y_train, test_size=.25, random_state=123)

In [13]:
#make it
tree = DecisionTreeClassifier(max_depth=4, random_state=123)

In [14]:
#fit and score
cross_val_score(tree, X_train, y_train, cv=5)

array([0.57142857, 0.65714286, 0.65714286, 0.57142857, 0.6       ])

## grid_search

In [16]:
#make our modeling object
knn = KNeighborsClassifier()

In [18]:
#define hyperparameters to send in
parameters = {'n_neighbors':range(1,21)}
parameters

{'n_neighbors': range(1, 21)}

In [19]:
#make the gridsearch object
grid = GridSearchCV(knn, parameters, cv=5)
grid

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': range(1, 21)})

In [22]:
#fit the gridsearch object
grid.fit(X_train, y_train)

/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be acce

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': range(1, 21)})

In [23]:
#use the gridsearch object
grid.cv_results_.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_n_neighbors', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])

In [24]:
grid.cv_results_['params']

[{'n_neighbors': 1},
 {'n_neighbors': 2},
 {'n_neighbors': 3},
 {'n_neighbors': 4},
 {'n_neighbors': 5},
 {'n_neighbors': 6},
 {'n_neighbors': 7},
 {'n_neighbors': 8},
 {'n_neighbors': 9},
 {'n_neighbors': 10},
 {'n_neighbors': 11},
 {'n_neighbors': 12},
 {'n_neighbors': 13},
 {'n_neighbors': 14},
 {'n_neighbors': 15},
 {'n_neighbors': 16},
 {'n_neighbors': 17},
 {'n_neighbors': 18},
 {'n_neighbors': 19},
 {'n_neighbors': 20}]

In [25]:
grid.cv_results_['mean_test_score']

array([0.59428571, 0.59428571, 0.58285714, 0.61714286, 0.57142857,
       0.59428571, 0.56      , 0.58285714, 0.57142857, 0.6       ,
       0.62285714, 0.60571429, 0.61714286, 0.62857143, 0.60571429,
       0.60571429, 0.58857143, 0.61714286, 0.6       , 0.63428571])

In [26]:
grid.best_estimator_

KNeighborsClassifier(n_neighbors=20)

In [29]:
grid.predict(X_validate)[:10]

/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


array(['auto', 'auto', 'auto', 'auto', 'auto', 'auto', 'manual', 'manual',
       'auto', 'manual'], dtype=object)

In [30]:
grid.predict_proba(X_validate)[:10]

array([[0.6 , 0.4 ],
       [0.65, 0.35],
       [0.9 , 0.1 ],
       [0.9 , 0.1 ],
       [0.55, 0.45],
       [0.5 , 0.5 ],
       [0.35, 0.65],
       [0.45, 0.55],
       [0.9 , 0.1 ],
       [0.45, 0.55]])

In [31]:
#save into varible to run on test data
knn_model = grid.best_estimator_
knn_model

KNeighborsClassifier(n_neighbors=20)

In [32]:
knn_model.score(X_test, y_test)

/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


0.6610169491525424

In [33]:
#make our modeling object
dt = DecisionTreeClassifier(random_state=123)

In [34]:
#define our parameters to rotate through
parameters = {
    'max_depth':range(1,21),
    'min_samples_leaf':range(1,11),
    'criterion': ["gini", "entropy", "log_loss"]
}

In [35]:
#make gridsearch object
grid = GridSearchCV(dt, parameters, cv=5)

In [36]:
X_train.shape

(131, 5)

In [37]:
#fit gridsearch object
grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=123),
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': range(1, 21),
                         'min_samples_leaf': range(1, 11)})

In [38]:
#use gridsearch object
len(grid.cv_results_['params'])

600

In [39]:
grid.cv_results_['params'][:15]

[{'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 1},
 {'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 2},
 {'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 3},
 {'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 4},
 {'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 5},
 {'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 6},
 {'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 7},
 {'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 8},
 {'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 9},
 {'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 10},
 {'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 1},
 {'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 2},
 {'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 3},
 {'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 4},
 {'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 5}]

In [40]:
grid.cv_results_['mean_test_score'][:15]

array([0.57179487, 0.57179487, 0.57179487, 0.57179487, 0.57179487,
       0.57179487, 0.57179487, 0.57179487, 0.57179487, 0.57179487,
       0.61794872, 0.63333333, 0.63333333, 0.63333333, 0.62564103])

In [41]:
for p, score in zip(grid.cv_results_['params'], grid.cv_results_['mean_test_score']):
    p['score'] = score

In [42]:
grid.cv_results_['params'][:5]

[{'criterion': 'gini',
  'max_depth': 1,
  'min_samples_leaf': 1,
  'score': 0.5717948717948718},
 {'criterion': 'gini',
  'max_depth': 1,
  'min_samples_leaf': 2,
  'score': 0.5717948717948718},
 {'criterion': 'gini',
  'max_depth': 1,
  'min_samples_leaf': 3,
  'score': 0.5717948717948718},
 {'criterion': 'gini',
  'max_depth': 1,
  'min_samples_leaf': 4,
  'score': 0.5717948717948718},
 {'criterion': 'gini',
  'max_depth': 1,
  'min_samples_leaf': 5,
  'score': 0.5717948717948718}]

In [43]:
pd.DataFrame(grid.cv_results_['params']).sort_values('score', ascending=False)

,criterion,max_depth,min_samples_leaf,score
370,entropy,18,1,0.686610
580,log_loss,19,1,0.686610
550,log_loss,16,1,0.686610
320,entropy,13,1,0.686610
520,log_loss,13,1,0.686610
...,...,...,...,...
238,entropy,4,9,0.565527
444,log_loss,5,5,0.564957
244,entropy,5,5,0.564957
435,log_loss,4,6,0.557550


In [44]:
dt_model = grid.best_estimator_
dt_model

DecisionTreeClassifier(criterion='entropy', max_depth=13, random_state=123)

In [45]:
#evaluate on test
dt_model.score(X_test, y_test)

0.7457627118644068